# Vectorización

## Librerias

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de Datos

In [3]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Transformación TF-IDF

In [4]:
tfidfvect = TfidfVectorizer()

X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


## Objetos y Funciones Auxiliares

In [5]:
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [6]:
def print_limited_strings(text, max_chars=200):
    # Imprime hasta 50 caracteres de cada texto en el dataset
    print(text[:max_chars])
    print('\n')  
    print('-' * max_chars)# Separador para distinguir entre textos


In [7]:
idx = np.random.randint(len(newsgroups_train.data))
print(idx)
print(newsgroups_train.data[idx])

9510
I recently had a case of shingles and my doctors wanted to give me
intravenous Acyclovir.

It was a pain finding IV sites in my arms...can I have some facts about
how advantageous it is to give intravenous antibiotics rather than oral?


# 1. Vectorizar documentos. 
Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

## Documento 1

In [8]:
idx = 968
print(newsgroups_train.data[idx])


You're right: Thomas, Gonzalez, Sheffield, and Griffey don't even begin
to compare with Ripken, Boggs, and Gwynn, so no wonder Alomar gets so
much attention.

Sandberg got no attention his rookie year because his rookie year was
terrible.  So was his sophomore year.

National League pitchers are "much better pitchers"?  That certainly explains
Sheffield's 1993, hm?  Are you confusing "have ERA's that are 0.40 lower
because they don't face DH's" with "much better"?


In [9]:
cossim = cosine_similarity(X_train[idx], X_train)[0]
cossim_values = np.sort(cossim)[::-1][1:6]
cossim_docs_idx = np.argsort(cossim)[::-1][1:6]

print(f'Id de documentos similares: {cossim_docs_idx}')
print(f'Valores de documentos similares: {cossim_values}')

Id de documentos similares: [5234 7528 1326 2510 8759]
Valores de documentos similares: [0.44382398 0.1898537  0.18803352 0.18744572 0.17540134]


In [10]:
target_original = newsgroups_train.target_names[y_train[idx]]
cossim_target_list = []
for i in cossim_docs_idx:
  cossim_target_list.append(newsgroups_train.target_names[y_train[i]])

print(f'Clase del documento original: {target_original}')
print(f'Clase de documentos similares: {cossim_target_list}')

Clase del documento original: rec.sport.baseball
Clase de documentos similares: ['rec.sport.baseball', 'rec.sport.baseball', 'rec.sport.baseball', 'rec.sport.baseball', 'rec.sport.hockey']


In [11]:
for i in cossim_docs_idx:
    print(f'Texto Número: {i}')
    print_limited_strings(newsgroups_train.data[i], max_chars=200)

Texto Número: 5234



Sandberg is not particulary known for his stolen bases.  What competition did 
Alomar have?  Sandberg came in a year after Ripken, and the same year as Boggs,
Gwynn, and the other magicians.  So le


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Texto Número: 7528
So far Simmons looks like a total idiot.


1) Zane Smith should learn how to "switchpitch" and return from the DL. I
would rather have Zane Smith pitch right handed than have Moeller pitch at all.

2)


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Texto Número: 1326
To all those out there wondering about who holds the record for three
homer games ina career, the answer is Johnny Mize in his career w

## Documento 2

In [12]:
idx = 266
print(newsgroups_train.data[idx])

........

It looks like the Edmonton Oilers just decided to take a European
vacation this spring...

Ranford, Tugnutt, Benning, Manson, Smith, Buchberger, and Corson
are playing for Canada.

Podein and Weight are playing for the US.

Is Kravchuk playing for the Russians...I know he had nagging
injuries late in the season.

Podein is an interesting case...because he was eligible to
play in Cape Breton in the AHL playoffs like Kovalev, Zubov,
and Andersson...obviously Sather and Pocklington are not
the total scrooges everyone makes them out to be...certainly
in this case they've massively outclassed Paramount and the
New York Rangers.


In [13]:
cossim = cosine_similarity(X_train[idx], X_train)[0]
cossim_values = np.sort(cossim)[::-1][1:6]
cossim_docs_idx = np.argsort(cossim)[::-1][1:6]

print(f'Id de documentos similares: {cossim_docs_idx}')
print(f'Valores de documentos similares: {cossim_values}')

Id de documentos similares: [4078 6165 9057  494 7942]
Valores de documentos similares: [0.23097329 0.2295853  0.21965255 0.19736223 0.19725845]


In [14]:
target_original = newsgroups_train.target_names[y_train[idx]]
cossim_target_list = []
for i in cossim_docs_idx:
  cossim_target_list.append(newsgroups_train.target_names[y_train[i]])

print(f'Clase del documento original: {target_original}')
print(f'Clase de documentos similares: {cossim_target_list}')

Clase del documento original: rec.sport.hockey
Clase de documentos similares: ['rec.sport.hockey', 'rec.sport.hockey', 'rec.sport.hockey', 'rec.sport.hockey', 'rec.sport.hockey']


In [15]:
for i in cossim_docs_idx:
    print(f'Texto Número: {i}')
    print_limited_strings(newsgroups_train.data[i], max_chars=200)

Texto Número: 4078
News reports in Toronto say that the Rangers are insisting that
Kovalev, Zubov, and Andersson play for Binghampton in the Calder
Cup playoffs, rather than return to play for their "home" countries
in 


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Texto Número: 6165
I can't believe that the NY Rangers would force its 
players to go to Binghamtom to play in the AHL playoffs instead
of letting them represent their countries in the World Championships.

Anderrson an


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Texto Número: 9057

Well, since the Oilers didn't win a heck of a lot of games, I'm not sure
they deserve and MVP ("can't win without him").  However, I'd

## Documento 3

In [16]:
idx = 3111
print(newsgroups_train.data[idx])

For some reason I never saw the original post on this thread, but if you are
looking for fast polygon routines on vga on a PC, you really can't go past
the mode X stuff from Dr Dobbs. This code is all p.domain (as far as I know),
and in the original articles, the routines were all presented as dumb vga
routines, and then optimised to modeX with some interesting discussion along
the way.
If you are interested, I could find out more details of the issues in question,
(I have them at home).


In [17]:
cossim = cosine_similarity(X_train[idx], X_train)[0]
cossim_values = np.sort(cossim)[::-1][1:6]
cossim_docs_idx = np.argsort(cossim)[::-1][1:6]

print(f'Id de documentos similares: {cossim_docs_idx}')
print(f'Valores de documentos similares: {cossim_values}')

Id de documentos similares: [ 3855 10996  2879  9623  6894]
Valores de documentos similares: [0.26018123 0.24266591 0.22628022 0.22581871 0.21231708]


In [18]:
target_original = newsgroups_train.target_names[y_train[idx]]
cossim_target_list = []
for i in cossim_docs_idx:
  cossim_target_list.append(newsgroups_train.target_names[y_train[i]])

print(f'Clase del documento original: {target_original}')
print(f'Clase de documentos similares: {cossim_target_list}')

Clase del documento original: comp.graphics
Clase de documentos similares: ['comp.graphics', 'comp.sys.mac.hardware', 'comp.sys.ibm.pc.hardware', 'talk.politics.mideast', 'talk.politics.guns']


In [19]:
for i in cossim_docs_idx:
    print(f'Texto Número: {i}')
    print_limited_strings(newsgroups_train.data[i], max_chars=300)

Texto Número: 3855
Hi there,

I've made a VGA mode 13h graphics library available via FTP.  I originally
wrote the routines as a kind of exercise for myself, but perhaps someone
here will find them useful.  They are certainly useable as they are, but
are missing some higher-level functionality.  They're intended more 


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Texto Número: 10996
I have MANY questions for all you experts out there pertaining to apple's
built-in video.  

#1, Do all macs that have built-in video have the ability to use VGA monitors?
#2, If so/if not, which macs have this capability?
#3, Can they drive SVGA as well?
#4, how big of a vga monitor can they drive?


------------------------------------------------------

## Documento 4

In [20]:
idx = 2954
print(newsgroups_train.data[idx])

The stragegy of the government is interesting.  The real fear comes from
them doing more than this.

This is a voluntary program, and thus harder for us to object to on
the surface.

Their strategy is a business one rather than legal one.  They are
pushing to get a standard in place, a secret standard, and if they
get it as a standard then they will drive competitors out of the market.
It will be legal to sell better, untapable encryption that doesn't have
registered keys, but it will be difficult, and thus not a plan for
most phone companies.

You see, with clipper chip phones you'll be able to talk to any
cellular company, or other phones or ports because they will follow
the standard.  AT&T has already announced a clipper chip encryption
product.  The government has marketed hard to get major vendors to
use these chips.   If they get enough market share, they will rule.

And thus there will be very little market for systems that can't be
tapped by the police.  The public isn't that 

In [21]:
cossim = cosine_similarity(X_train[idx], X_train)[0]
cossim_values = np.sort(cossim)[::-1][1:6]
cossim_docs_idx = np.argsort(cossim)[::-1][1:6]

print(f'Id de documentos similares: {cossim_docs_idx}')
print(f'Valores de documentos similares: {cossim_values}')

Id de documentos similares: [ 8534 10575  6806  2551  5856]
Valores de documentos similares: [0.40508258 0.39032881 0.38777565 0.3780144  0.37469965]


In [22]:
target_original = newsgroups_train.target_names[y_train[idx]]
cossim_target_list = []
for i in cossim_docs_idx:
  cossim_target_list.append(newsgroups_train.target_names[y_train[i]])

print(f'Clase del documento original: {target_original}')
print(f'Clase de documentos similares: {cossim_target_list}')

Clase del documento original: sci.crypt
Clase de documentos similares: ['sci.crypt', 'sci.crypt', 'sci.crypt', 'sci.crypt', 'sci.crypt']


In [23]:
for i in cossim_docs_idx:
    print(f'Texto Número: {i}')
    print_limited_strings(newsgroups_train.data[i], max_chars=200)

Texto Número: 8534
<If Clipper comes to cellular phones along with legal proscriptions against
<using other cipher systems on these phones, a new and potentially dangerous
<class of crime is created.

Aside from possess


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Texto Número: 10575

I am not an expert in the cryptography science, but some basic things
seem evident to me, things which this Clinton Clipper do not address.
The all pertain to opportunites for abuse, and conclusions 


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Texto Número: 6806

This isn't true.  Today's criminals regularly use all sorts of unsafe
methods, from cordless phones to cellular phones to plain old c

## Documento 5

In [24]:
idx = 9759
print(newsgroups_train.data[idx])


I would guess not.  Dr. Neil Gehrels of CGRO is the son of Dr. Tom
Gehrels of the University of Arizona.  Since he's long had research
interests in asteroids and other solar-system astronomy, Tom is the
one more likely to have discovered  a comet (and thus had his name
attached to it).

Tom Gehrels is a leader in the Spacewatch project, which has recently
increased mankind's discovery rate on near-Earth asteroids (they're
finding a couple every month).  For much more on this interesting guy,
read his autobiography, *On a Glassy Sea*.


In [25]:
cossim = cosine_similarity(X_train[idx], X_train)[0]
cossim_values = np.sort(cossim)[::-1][1:6]
cossim_docs_idx = np.argsort(cossim)[::-1][1:6]

print(f'Id de documentos similares: {cossim_docs_idx}')
print(f'Valores de documentos similares: {cossim_values}')

Id de documentos similares: [10594  1402  8613 10295  8610]
Valores de documentos similares: [0.51540546 0.28077187 0.23025051 0.17873329 0.17773624]


In [26]:
target_original = newsgroups_train.target_names[y_train[idx]]
cossim_target_list = []
for i in cossim_docs_idx:
  cossim_target_list.append(newsgroups_train.target_names[y_train[i]])

print(f'Clase del documento original: {target_original}')
print(f'Clase de documentos similares: {cossim_target_list}')

Clase del documento original: sci.space
Clase de documentos similares: ['sci.space', 'sci.space', 'sci.space', 'sci.space', 'comp.os.ms-windows.misc']


In [37]:
for i in cossim_docs_idx:
    print(f'Texto Número: {i}')
    print_limited_strings(newsgroups_train.data[i], max_chars=300)

Texto Número: 10594

Neil Gehrels is Prof. Tom Gehrels son.  Tom Gehrels was the discoverer
of P/Gehrels 3 (as well as about 4 other comets - the latest of which
does not bear his name, but rather the name "Spacewatch" since he was
observing with that system when he found the latest comet).  


------------------------


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Texto Número: 1402



No. I estimate a 99 % probability the Gehrels referred to
is Thomas Gehrels of the Spacewatch project, Kitt Peak observatory.

Maybe in the 24th century they could do gamma ray spectroscopy on
distant asteroids with an orbiting observatory, but here in the
primitive 20th we have to send a probe t


------------------------------------------------------

## Conclusiones

**Análisis de etiquetas:**
1. Tras realizar el análisis de similaridad entre documentos, seleccionando 5 documentos al azar y comparándolos con el resto de la colección, se ha observado que, en la mayoría de los casos (documentos 1, 2, 4 y 5), los textos más similares presentan una coherencia significativa con la etiqueta de clasificación original. Esto sugiere que el modelo de similaridad utilizado es generalmente efectivo para identificar contenidos relacionados dentro de la misma categoría temática.
2. Sin embargo, el análisis del tercer documento revela una excepción notable. En este caso, los textos más similares pertenecen a diferentes etiquetas de clasificación, algunas de ellas bastante divergentes en términos temáticos, como "talk.politics.guns" o "talk.politics.mideast". Esta discrepancia puede indicar la presencia de factores externos o una ambigüedad en el contenido que no fue captada por el modelo de similaridad.

**Análisis de contenido del texto:**
- _Documento 1:_
Todos los documentos proporcionados comparten el tema central del béisbol y contienen discusiones sobre el desempeño, estadísticas y comparaciones entre jugadores. La similitud es más fuerte en los documentos 1 y 4 debido a la mención directa de jugadores comunes y debates similares. Los documentos 2, 3 y 5 también son similares en contexto y contenido, aunque se enfocan en diferentes aspectos y jugadores del deporte.

- _Documento 2:_
Los textos proporcionados comparten un tema central relacionado con el hockey, específicamente sobre decisiones y políticas respecto a la participación de jugadores en diferentes ligas y competencias internacionales. La discusión gira en torno a jugadores específicos, equipos como los Edmonton Oilers y los New York Rangers, y la controversia sobre si los jugadores deben participar en los playoffs de ligas menores o representar a sus países en campeonatos mundiales.

- _Documento 3:_
Los textos proporcionados muestran una clara división en cuanto a los temas discutidos. Los primeros tres textos están centrados en aspectos técnicos relacionados con computadoras, específicamente en gráficos VGA, programación y preguntas sobre hardware. Estos textos, junto con el original, son similares entre sí, ya que todos tratan de temas técnicos y de desarrollo informático.
Sin embargo, los últimos dos textos (4 y 5) no tienen relación con el tema técnico de los primeros. El texto 4 parece abordar cuestiones de derechos humanos en un contexto geopolítico, mientras que el texto 5 es un comunicado de prensa de la Casa Blanca, lo cual es completamente distinto del enfoque técnico de los primeros documentos.

- _Documento 4:_
Los textos proporcionados comparten un enfoque común relacionado con la criptografía, la seguridad en las telecomunicaciones y las implicaciones del uso de tecnologías como el Clipper Chip. La mayoría de los textos discuten preocupaciones sobre la privacidad, la seguridad, la intervención del gobierno y las posibles consecuencias de implementar estándares de cifrado que podrían ser explotados o utilizados para fines de vigilancia.
En conjunto, estos textos son similares en cuanto al tema central, ya que todos abordan aspectos técnicos y éticos de la criptografía y la seguridad en las comunicaciones. La preocupación sobre el control gubernamental y las implicaciones para la privacidad son temas recurrentes en estos documentos, lo que los hace altamente relacionados y coherentes entre sí.

- _Documento 5:_
Los textos proporcionados en su mayoría comparten un enfoque común relacionado con la figura de Tom Gehrels y sus contribuciones al campo de la astronomía, específicamente en el descubrimiento de cometas y su liderazgo en el proyecto Spacewatch. Los documentos discuten su legado, la conexión con su hijo Neil Gehrels, y mencionan su trabajo en la astronomía del sistema solar.
Los textos 1 a 3 son altamente similares, ya que todos se centran en la misma persona y su trabajo en el ámbito astronómico. Los textos 4 y 5, aunque están relacionados con temas científicos y tecnológicos, se desvían significativamente del tema central de los primeros tres, enfocándose en cálculos matemáticos de órbitas y en la búsqueda de información de contacto de otra persona.

# 2. Clasificación Naïve Bayes
- Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos
- Probar modelos de Naïve Bayes Multinomial y ComplementNB.

## Multinomial

In [28]:
# Crear un pipeline con TfidfVectorizer y MultinomialNB
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

# Definir el grid de parámetros para GridSearchCV
param_grid = {
    'tfidf__max_df': [0.1, 0.3, 0.5],
    'tfidf__ngram_range': [(1, 2)],
    'tfidf__lowercase': [True, False],
    'nb__alpha': np.arange(0.01, 0.016, 0.005),
    'nb__fit_prior': [False]
}

# Crear el scorer para F1 con average='macro'
f1_macro_scorer = make_scorer(f1_score, average='macro')

# Configurar el GridSearchCV
multinomial = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2, scoring=f1_macro_scorer)

In [29]:
multinomial.fit(newsgroups_train.data, newsgroups_train.target)
print("\n Mejores Parametros Encontrados:")
print(multinomial.best_params_)
print("\n Mejor puntaje de cross-validation:")
print(multinomial.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits

 Mejores Parametros Encontrados:
{'nb__alpha': np.float64(0.01), 'nb__fit_prior': False, 'tfidf__lowercase': True, 'tfidf__max_df': 0.1, 'tfidf__ngram_range': (1, 2)}

 Mejor puntaje de cross-validation:
0.7488635630581871


In [30]:
y_pred =  multinomial.predict(newsgroups_test.data)
f1_score(y_test, y_pred, average='macro')

np.float64(0.6750288007499659)

## ComplementNB

In [31]:
# Crear un pipeline con TfidfVectorizer y MultinomialNB
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', ComplementNB())
])

# Definir el grid de parámetros para GridSearchCV
param_grid = {
    'tfidf__max_df': [0.1, 0.3, 0.5],
    'tfidf__ngram_range': [(1, 2)],
    'tfidf__lowercase': [True, False],
    'nb__alpha': np.arange(0.01, 0.016, 0.005),
    'nb__fit_prior': [False]
}

# Crear el scorer para F1 con average='macro'
f1_macro_scorer = make_scorer(f1_score, average='macro')

# Configurar el GridSearchCV
complement = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=2, scoring=f1_macro_scorer)

In [32]:
complement.fit(newsgroups_train.data, newsgroups_train.target)
print("\n Mejores Parametros Encontrados:")
print(complement.best_params_)
print("\n Mejor puntaje de cross-validation:")
print(complement.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits

 Mejores Parametros Encontrados:
{'nb__alpha': np.float64(0.015), 'nb__fit_prior': False, 'tfidf__lowercase': False, 'tfidf__max_df': 0.3, 'tfidf__ngram_range': (1, 2)}

 Mejor puntaje de cross-validation:
0.7621714472506199


In [33]:
y_pred =  complement.predict(newsgroups_test.data)
f1_score(y_test, y_pred, average='macro')

np.float64(0.6921568285995371)

## Conclusiones

Se entrenaron y evaluaron dos modelos de clasificación Naïve Bayes: Multinomial y ComplementNB, con el objetivo de maximizar el desempeño de clasificación (f1-score macro).
El modelo ComplementNB mostró un rendimiento superior tanto en la validación cruzada como en el conjunto de test en comparación con el modelo Multinomial. 

En particular:
- Multinomial Naïve Bayes alcanzó un puntaje de f1 en el conjunto de test de 0.6750 y un puntaje en validación cruzada de 0.7489.
- ComplementNB logró un mejor puntaje de f1 en el conjunto de test de 0.6922 y un mejor puntaje en validación cruzada de 0.7622.

Estos resultados indican que, bajo las configuraciones de parámetros seleccionadas, el modelo ComplementNB no solo generaliza mejor. 

La diferencia en los parámetros, como el uso de minúsculas y la proporción máxima de documentos (max_df), junto con un ajuste fino del parámetro alpha, ha contribuido a mejorar el desempeño de ComplementNB en este caso específico.

# 3. Transponer matriz documento-término
Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [34]:
transposed_x = X_train.T
transposed_x

<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 1103627 stored elements and shape (101631, 11314)>

In [35]:
selected_words = ['century', 'explosion', 'religion', 'football', 'jobs']
indexes = [tfidfvect.vocabulary_[word] for word in selected_words]
indexes

[26461, 38788, 77274, 41101, 52493]

In [36]:
for i, word in enumerate(selected_words):
    print(f'Palabra: {word}')
    cossim = cosine_similarity(transposed_x[indexes[i]], transposed_x)[0]
    cossim_values = np.sort(cossim)[::-1][1:6]
    cossim_words_idx = np.argsort(cossim)[::-1][1:6]
    similar_words = [tfidfvect.get_feature_names_out()[idx] for idx in cossim_words_idx]
    print(f'Palabras similares: {similar_words}')
    print(f'Valores de similitud de palabras: {cossim_values}')
    print('-'*100)

Palabra: century
Palabras similares: ['20th', 'autographs', '1909', '1883', 'homework']
Valores de similitud de palabras: [0.40049998 0.3256703  0.30023232 0.29808301 0.25549192]
----------------------------------------------------------------------------------------------------
Palabra: explosion
Palabras similares: ['ignited', 'leveled', 'elevator', 'grain', 'mehola']
Valores de similitud de palabras: [0.68873205 0.61372846 0.58244308 0.49827317 0.31902375]
----------------------------------------------------------------------------------------------------
Palabra: religion
Palabras similares: ['religious', 'religions', 'categorized', 'purpsoe', 'crusades']
Valores de similitud de palabras: [0.24507616 0.2116499  0.20391513 0.20075281 0.19870798]
----------------------------------------------------------------------------------------------------
Palabra: football
Palabras similares: ['madden', 'palying', 'zabriskie', 'sportstalk', 'basketball']
Valores de similitud de palabras: [0.37

## Conclusiones

1. Palabra: "century"
- Palabras similares: ['20th', 'autographs', '1909', '1883', 'homework']
- Conclusión: Las palabras más similares a "century" están mayormente relacionadas con periodos de tiempo específicos ('20th', '1909', '1883'), lo cual es coherente dado que "century" es un término temporal. La presencia de 'autographs' sugiere una posible conexión con eventos históricos o coleccionismo, mientras que 'homework' parece ser menos directamente relacionado, quizás indicando un uso menos común o una interpretación más amplia del contexto en el que se encuentran estas palabras.

2. Palabra: "explosion"
- Palabras similares: ['ignited', 'leveled', 'elevator', 'grain', 'mehola']
- Conclusión: Las palabras más similares a "explosion" son altamente coherentes en relación con eventos de explosiones y sus posibles causas o efectos. 'Ignited' y 'leveled' están directamente relacionados con la idea de una explosión. 'Elevator' y 'grain' pueden sugerir contextos como explosiones en silos de grano, mientras que 'mehola' parece ser un término menos interpretado, posiblemente relacionado con un evento específico o un lugar asociado a una explosión.

3. Palabra: "religion"
- Palabras similares: ['religious', 'religions', 'categorized', 'purpsoe', 'crusades']
- Conclusión: Las palabras similares a "religion" son muy coherentes y refuerzan el tema. 'Religious' y 'religions' están directamente relacionadas, mientras que 'categorized' podría reflejar una organización o clasificación de religiones. 'Purpsoe' parece ser una variante ortográfica de 'purpose', que podría relacionarse con los objetivos o motivaciones de la religión, y 'crusades' conecta directamente con un evento histórico asociado a la religión.

4. Palabra: "football"
- Palabras similares: ['madden', 'palying', 'zabriskie', 'sportstalk', 'basketball']
- Conclusión: Las palabras similares a "football" están claramente relacionadas con el deporte, en particular con referencias culturales ('madden', que podría referirse a los videojuegos de Madden NFL) y otros deportes ('basketball'). 'Palying' parece ser una variante de 'playing', relacionada con el acto de jugar al fútbol. 'Zabriskie' podría ser un apellido de un jugador o una referencia dentro del contexto deportivo, y 'sportstalk' sugiere discusiones sobre deportes, lo cual es coherente con la temática.

5. Palabra: "jobs"
- Palabras similares: ['jobless', 'paring', 'toughen', 'chalk', 'filibustering']
- Conclusión: Las palabras similares a "jobs" se relacionan principalmente con el mercado laboral y los desafíos asociados. 'Jobless' está directamente relacionado con la falta de empleo. 'Paring' y 'toughen' podrían estar relacionados con acciones de ajuste o endurecimiento en el mercado laboral. 'Chalk' y 'filibustering', aunque menos evidentes, podrían estar conectados con discusiones políticas o educativas sobre empleo, o ser términos utilizados en contextos específicos dentro de este tema.